In [27]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import time
import random

In [28]:
X,y = load_diabetes(return_X_y=True)

In [29]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [31]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

In [32]:
print(lr.coef_)
print(lr.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [33]:
X_train.shape

(353, 10)

In [34]:
! python --version

Python 3.12.12


In [35]:
class GDRegressor:
  def __init__(self, learning_rate=0.01, epochs=100):
    self.coef_ = None
    self.intercept_ = None
    self.lr = learning_rate
    self.epochs = epochs
    self.cost_history = []
  def fit(self,X_train,y_train):
    self.intercept_ = 0
    self.coef_ = np.ones(X_train.shape[1])
    m = X_train.shape[0]
    for _ in range(self.epochs):
      y_hat = X_train @ self.coef_ + self.intercept_
      error = y_hat - y_train

      dj_dw = (1/m)*(X_train.T @ error)
      dj_db = (1/m)*(np.sum(error))

      self.intercept_ -= self.lr * dj_db
      self.coef_ -= self.lr * dj_dw

      cost = (1/(2*m))*np.sum(error**2)
      self.cost_history.append(cost)
    return self

  def predict(self, X_test):
    prediction = X_test @ self.coef_ + self.intercept_
    return prediction

In [36]:
lr = GDRegressor(learning_rate=0.5, epochs=353*4)
start = time.time()
lr.fit(X_train,y_train)
print(time.time()-start)

0.034417152404785156


In [37]:
y_pred = lr.predict(X_test)

In [38]:
print(lr.coef_)
print(lr.intercept_)

[  29.30377876 -140.94465099  453.19033182  305.006973    -23.15191084
  -92.24274815 -191.88097044  113.70604857  410.22257192  113.4590216 ]
152.06582548861158


In [39]:
r2_score(y_test, y_pred)

0.4532892909681887

In [40]:
class SGDRegressor:
  def __init__(self, learning_rate=0.01, epochs=100):
    self.coef_ = None
    self.intercept_ = None
    self.lr = learning_rate
    self.epochs = epochs

  def fit(self,X_train,y_train):
    self.intercept_ = 0
    self.coef_ = np.ones(X_train.shape[1])
    m = X_train.shape[0]
    for _ in range(self.epochs):
      for j in range(X_train.shape[0]):
        idx = np.random.randint(0, X_train.shape[0])
        y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
        error = y_hat - y_train[idx]

        dj_dw = (2)*np.dot(X_train[idx], error)
        dj_db = (2)*(error)

        self.intercept_ -= self.lr * dj_db
        self.coef_ -= self.lr * dj_dw

    return self

  def predict(self, X_test):
    prediction = (X_test @ self.coef_) + self.intercept_
    return prediction

In [41]:
sgd = SGDRegressor(learning_rate=0.5, epochs=4)
start = time.time()
sgd.fit(X_train,y_train)
print(time.time()-start)

0.025037288665771484


In [42]:
y_pred2 = sgd.predict(X_test)

In [43]:
r2_score(y_test,y_pred2)

0.3619209633460001

Mini batch GD

In [44]:
class MBGDRegressor:
  def __init__(self, batch_size=10, learning_rate=0.01, epochs=50):
    self.batch_size = batch_size
    self.lr = learning_rate
    self.epochs = epochs
    self.coef_ = None
    self.intercept_ = None

  def fit(self, X_train, y_train):
    # initialize values
    self.intercept_ = 1
    self.coef_ = np.zeros(X_train.shape[1])

    for i in range(self.epochs):

      for j in range(int(X_train.shape[0]/self.batch_size)):
        idx = random.sample(range(0, X_train.shape[0]), self.batch_size)
        y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_

        dj_db = 2*(np.mean(y_hat - y_train[idx]))
        self.intercept_ -= self.lr*(dj_db)

        dj_dw = 2*(np.dot((y_hat - y_train[idx]), X_train[idx]))
        self.coef_ -= self.lr*(dj_dw)
    return self

  def predict(self, X_test):
    return (X_test @ self.coef_) + self.intercept_

In [45]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/25), learning_rate=0.03, epochs=50)
mbr.fit(X_train,y_train)

In [46]:
y_pred = mbr.predict(X_test)
r2_score(y_test, y_pred)

0.45039061255057133